# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Capstone Project - Malay Language Sentiment Analysis

## Part 4 - Deployment

In this notebook, we will go through the steps to:
1. Create a Flask API based on the best model
2. Create a Dockerfile
3. Deploy to Google Cloud
4. Host on Streamlit

## 1. Flask API

In [1]:
%%writefile sentiment.py 
from flask import Flask, request, jsonify
import os
import pickle
import numpy as np

#import your_nlp_library as nlp

app = Flask(__name__)

# Load your NLP model here (replace with actual loading code)
# Example:
with open('03_production_sentiment_model.pkl', 'rb') as file:
    # Unpickle the object
    model = pickle.load(file)

@app.route('/predict_sentiment', methods=['POST'])
def predict_sentiment():
    try:
        data = request.get_json()
        sentences = data['sentences']

        # Join the sentences into a single string
        input_text = ' '.join(sentences)

        # Get probabilities for each class
        prediction_proba = model.predict_proba([input_text])[0]

        # Determine the class based on the highest probability
        class_labels = ['negative', 'neutral', 'positive']
        sentiment = class_labels[np.argmax(prediction_proba)]

        # Round the probabilities to 2 significant figures
        probabilities = [round(prob, 2) for prob in prediction_proba]

        response = {'sentiment': sentiment, 'probability': probabilities}
        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__': # good practise to have this main block whenever creating a .py file
    app.run(host='0.0.0.0', # run the 'api' object created above with 2 routes on local host url '0.0.0.0' to just run on this computer
            debug=True, # Debug=True ensures any changes to inference.py (like adding an extra print somewhere in this script) automatically updates the running API
            port=int(os.environ.get("PORT", 8081)) # just use 8080 by default. This 8080 runs the API locally
           ) 

Writing sentiment.py


At this point, open a terminal in this folder and enter:

```bash
mamba activate malay_sentiment_project
```
and

```bash
python sentiment.py
```

<mark>**Note:**</mark>This is based on the virtual environment we created in Notebook 01. If you created a virtual environment that is named differently, mamba activate that virtual environment.

Then, run the code cell below to test it.

In [19]:
#Testing script

import requests

# Define the URL of your Flask app
url = 'http://127.0.0.1:8081/predict_sentiment'  # Replace with the actual URL

# Define the input data as a dictionary
data = {
    'sentences': [
        'kau dh knp ni?'
    ]
}

# Send a POST request to the Flask app
response = requests.post(url, json=data)

# Check if the request was successful
if response.status_code == 200:
    result = response.json()  # Parse the JSON response
    print('Sentiment prediction:', result)
else:
    print('Error:', response.status_code, response.text)


Sentiment prediction: {'probability': [0.6, 0.24, 0.16], 'sentiment': 'negative'}


## 2. Create Dockerfile

Now that we know that the API works, create a Dockerfile to put on Google Cloud so that the API is hosted there.

In [12]:
%%writefile Dockerfile
# Use the official Python image
FROM python:3.8-slim

# Set the working directory
WORKDIR /app

# Copy your Flask application and model file
COPY sentiment.py .
COPY 03_production_sentiment_model.pkl .

# Install any necessary Python libraries for your Flask app
RUN pip install flask joblib requests scikit-learn pandas numpy imbalanced-learn

# Expose the port your Flask app will run on
EXPOSE 8081

# Run the Flask app
CMD ["python", "sentiment.py"]

Writing Dockerfile


## 3. Deploy to Google Cloud

At this point, open a terminal in this folder and run:

```bash
gcloud run deploy malay-sentiment-analysis --source . --region asia-southeast1
```
<mark>**Note:**</mark> Choose a region of your choice.

Type y to ANY message you get on your terminal and press 'return'.

In [20]:
# Testing the cloud-hosted API
import requests

api_url = 'https://malay-sentiment-analysis-nqaybcgjca-as.a.run.app'
api_route = '/predict_sentiment'

user_input = {'sentences': ['kau dh knp ni?']} 

response = requests.post(f'{api_url}{api_route}', json = user_input)
predictions = response.json()

print(predictions)

{'probability': [0.6, 0.24, 0.16], 'sentiment': 'negative'}


## 4. Host on Streamlit

Finally, host it on Streamlit. 
1. Run the code below
2. Copy the 'malay_sentiment_analysis_streamlit.py' script to your relevant GitHub folder
3. git push to your personal GitHub
4. Sign up for [Streamlit Cloud](https://streamlit.io/cloud) and connect it to your personal GitHub
5. Create a 'New app' from existing repo and ensure that the repository, branch, and main file path point towards the 'malay_sentiment_analysis_streamlit.py' script

In [2]:
%%writefile malay_sentiment_analysis_streamlit.py
import streamlit as st
import requests
import json

#st.set_page_config(page_title="Alat Analisis Sentiment Media Sosial")
st.set_page_config(page_title="Social Media Sentiment Analysis Tool")

# Title of the webpage
#st.title("Alat Analisis Sentiment Media Sosial 📊👥💬")
st.title("Social Media Sentiment Analysis Tool (Bahasa Melayu) 📊👥💬")

# Create an empty container to hold the user input
user_input_container = st.empty()

# Get user inputs
sentence = user_input_container.text_area(label='Input a sentence in Malay to determine its sentiment. 🇸🇬🇲🇾🇧🇳🇮🇩')
#sentence = user_input_container.text_area(label='Isi ayat dalam Bahasa Melayu untuk menentukan sentimen ayat tersebut. 🇸🇬🇲🇾🇧🇳🇮🇩')

# Display the inputs
user_input = {"sentences": [sentence]} 
st.write("User input:")
st.write(user_input)

# Code to post the user inputs to the API and get the predictions
# Paste the URL to your GCP Cloud Run API here!
api_url = 'https://malay-sentiment-analysis-nqaybcgjca-as.a.run.app'  # replace with own Google Cloud Run API
api_route = '/predict_sentiment'

# Add a submit button
if st.button("Submit"):  # only display model predictions on UI if user clicks "Submit" button
    response = requests.post(f'{api_url}{api_route}', json=user_input)
    predictions = response.json()  # return dictionary with key 'predictions' & values are a list of predictions
    
    st.write(f"Sentiment: {predictions['sentiment']}")  # prediction values were stored in 'predictions' key of dict: predictions. [0] is to give prediction output 0/1 in a "unlisted" format since we're only sending user inputs for 1 row of X at a time
   # # Extract the sentiment probability from the dictionary
   #  negative_probability = predictions["probability"][0]
   #  neutral_probability = predictions["probability"][1]
   #  positive_probability = predictions["probability"][2]

   #  # Create a dictionary to pass to st.bar_chart and round the values
   #  chart_data = {
   #      "Sentiment": ["Negative", "Neutral", "Positive],
   #      "Probability": [negative_probability, neutral_probability, positive_probability]
   #  }

   #  # Create a bar chart
   #  st.bar_chart(chart_data, x="Sentiment", y="Probability", use_container_width=True)

Overwriting malay_sentiment_analysis_streamlit.py
